In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kolaboti2/img_160.jpg
/kaggle/input/kolaboti2/img_2104.jpg
/kaggle/input/kolaboti2/img_1306.jpg
/kaggle/input/kolaboti2/img_233.jpg
/kaggle/input/kolaboti2/img_1466.jpg
/kaggle/input/kolaboti2/img_1410.jpg
/kaggle/input/kolaboti2/img_1025.jpg
/kaggle/input/kolaboti2/img_359.jpg
/kaggle/input/kolaboti2/img_684.jpg
/kaggle/input/kolaboti2/img_1479.jpg
/kaggle/input/kolaboti2/img_801.jpg
/kaggle/input/kolaboti2/img_2121.jpg
/kaggle/input/kolaboti2/img_1689.jpg
/kaggle/input/kolaboti2/img_470.jpg
/kaggle/input/kolaboti2/img_516.jpg
/kaggle/input/kolaboti2/img_645.jpg
/kaggle/input/kolaboti2/img_1818.jpg
/kaggle/input/kolaboti2/img_21.jpg
/kaggle/input/kolaboti2/img_1721.jpg
/kaggle/input/kolaboti2/img_1548.jpg
/kaggle/input/kolaboti2/img_1151.jpg
/kaggle/input/kolaboti2/img_699.jpg
/kaggle/input/kolaboti2/img_1812.jpg
/kaggle/input/kolaboti2/img_289.jpg
/kaggle/input/kolaboti2/img_1402.jpg
/kaggle/input/kolaboti2/img_693.jpg
/kaggle/input/kolaboti2/img_572.jpg
/kaggle/input/k

In [2]:
from pathlib import Path

path = Path("/kaggle/input")

print("Number of images in each category:\n")
for directory in path.iterdir():
    print(f"{directory.name}: {len(list(directory.iterdir()))}")

Number of images in each category:

kolaboti2: 2440
four-oclock2: 2540
zinnia2: 2572
kathgulap2: 2574
rangoon-creeper2: 2732
inception-v3: 1
catharanthus2: 2576
rainlily2: 755


In [3]:
import pandas as pd
from pathlib import Path

def split_dataset(path, training_pct=0.70, validation_pct=0.20):
    train_df = pd.DataFrame({'path': [], 'category': []})
    test_df = pd.DataFrame({'path': [], 'category': []})
    valid_df = pd.DataFrame({'path': [], 'category': []})
    
    for category_path in path.iterdir():
        if not category_path.is_dir():  # Skip non-directory files
            continue
            
        category_name = category_path.name
        images_files = list(category_path.glob('*.*'))  # Get all files in directory
        
        # Skip empty categories
        if not images_files:
            continue
            
        num_train = int(len(images_files) * training_pct)
        num_valid = int(len(images_files) * validation_pct)

        # Split the images
        training_images = images_files[:num_train]
        valid_images = images_files[num_train:(num_train+num_valid)]
        test_images = images_files[(num_train+num_valid):]

        # Create DataFrames for this category's splits
        temp_train = pd.DataFrame({
            'path': [str(img) for img in training_images],
            'category': category_name
        })
        temp_valid = pd.DataFrame({
            'path': [str(img) for img in valid_images],
            'category': category_name
        })
        temp_test = pd.DataFrame({
            'path': [str(img) for img in test_images],
            'category': category_name
        })
        
        # Concatenate with the main DataFrames
        train_df = pd.concat([train_df, temp_train], ignore_index=True)
        valid_df = pd.concat([valid_df, temp_valid], ignore_index=True)
        test_df = pd.concat([test_df, temp_test], ignore_index=True)
    
    return train_df, valid_df, test_df

# Example usage:
path = Path("/kaggle/input")  # Replace with your actual dataset path
train_df, valid_df, test_df = split_dataset(path)

# Print the results
print(f"Training set: {len(train_df)} samples")
print(f"Validation set: {len(valid_df)} samples")
print(f"Test set: {len(test_df)} samples")
print("\nFirst few training samples:")
print(train_df.head())

Training set: 11330 samples
Validation set: 3236 samples
Test set: 1623 samples

First few training samples:
                                   path   category
0   /kaggle/input/kolaboti2/img_160.jpg  kolaboti2
1  /kaggle/input/kolaboti2/img_2104.jpg  kolaboti2
2  /kaggle/input/kolaboti2/img_1306.jpg  kolaboti2
3   /kaggle/input/kolaboti2/img_233.jpg  kolaboti2
4  /kaggle/input/kolaboti2/img_1466.jpg  kolaboti2


In [4]:
train_df.head()

,path,category
0,/kaggle/input/kolaboti2/img_160.jpg,kolaboti2
1,/kaggle/input/kolaboti2/img_2104.jpg,kolaboti2
2,/kaggle/input/kolaboti2/img_1306.jpg,kolaboti2
3,/kaggle/input/kolaboti2/img_233.jpg,kolaboti2
4,/kaggle/input/kolaboti2/img_1466.jpg,kolaboti2


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-05-06 08:15:52.094649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746519352.297586      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746519352.357112      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
train_data = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.20,
    height_shift_range=0.20,
    shear_range=0.20,
    horizontal_flip=True,
)

train_datagen = train_data.flow_from_dataframe(
    train_df,
    target_size=(200, 200),
    x_col='path',
    y_col='category',
    batch_size=32,
    class_mode='categorical',
)

Found 11330 validated image filenames belonging to 7 classes.


In [7]:
validation_data = ImageDataGenerator()
validation_datagen = validation_data.flow_from_dataframe(
    valid_df,
    batch_size=32,
    x_col='path',
    y_col='category'
   
)

Found 3236 validated image filenames belonging to 7 classes.


In [8]:
test_data = ImageDataGenerator()
test_datagen = validation_data.flow_from_dataframe(
    test_df,
    x_col='path',
    y_col='category'
)

Found 1623 validated image filenames belonging to 7 classes.


In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop

In [10]:
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPool2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1746519382.773004      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746519382.773703      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 198, 198, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 99, 99, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 97, 97, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 48, 48, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 46, 46, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 23, 23, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 33856)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       4,333,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,366,186 (16.66 MB)

 Trainable params: 4,366,186 (16.66 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

# 1. Prepare Image Data Generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input',  # <-- Replace this
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    '/kaggle/input',  # <-- Replace this
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# 2. Define a compatible CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),  # Output shape depends on image size and conv layers
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # output layer
])

# 3. Compile the model
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 4. Verify class info
print(f"Detected classes: {train_generator.num_classes}")
print(f"Class indices: {train_generator.class_indices}")

# 5. Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)


Found 16189 images belonging to 8 classes.
Found 16189 images belonging to 8 classes.
Detected classes: 8
Class indices: {'catharanthus2': 0, 'four-oclock2': 1, 'inception-v3': 2, 'kathgulap2': 3, 'kolaboti2': 4, 'rainlily2': 5, 'rangoon-creeper2': 6, 'zinnia2': 7}
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1746519396.326840      98 service.cc:148] XLA service 0x799a4400eca0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746519396.327621      98 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1746519396.327648      98 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1746519396.568759      98 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/506 ━━━━━━━━━━━━━━━━━━━━ 1:09:03 8s/step - accuracy: 0.1562 - loss: 2.0723

I0000 00:00:1746519400.146696      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


506/506 ━━━━━━━━━━━━━━━━━━━━ 184s 348ms/step - accuracy: 0.4592 - loss: 1.6587 - val_accuracy: 0.8076 - val_loss: 0.5743
Epoch 2/5
506/506 ━━━━━━━━━━━━━━━━━━━━ 0s 105us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/5


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


506/506 ━━━━━━━━━━━━━━━━━━━━ 134s 263ms/step - accuracy: 0.7809 - loss: 0.6434 - val_accuracy: 0.8872 - val_loss: 0.3292
Epoch 4/5
506/506 ━━━━━━━━━━━━━━━━━━━━ 0s 61us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/5
506/506 ━━━━━━━━━━━━━━━━━━━━ 131s 257ms/step - accuracy: 0.8657 - loss: 0.4092 - val_accuracy: 0.8227 - val_loss: 0.4963


In [ ]:
import tensorflow_hub as hub
    
keras_layer = hub.KerasLayer('https://kaggle.com/models/google/inception-v3/frameworks/TensorFlow2/variations/classification/versions/2')

In [13]:
pip install matplotlib


Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [20]:
from PIL import Image
import os

# Your flat image folder
image_dir = "/kaggle/input/inception-v3"
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

# Load one image using PIL
img_path = os.path.join(image_dir, image_files[3])
image = Image.open(img_path)
image = transform(image)

print("Image shape:", image.shape)


IndexError: list index out of range

In [18]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define path to image directory
data_dir = "/kaggle/input"  # adjust this to your image dataset path

# Define image transforms (resizing, normalization, etc.)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Load dataset from directory
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Check class names (e.g., ['churn', 'no_churn'])
print("Classes:", dataset.classes)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example: Get one batch
images, labels = next(iter(dataloader))
print("Image batch shape:", images.shape)
print("Label batch shape:", labels.shape)


FileNotFoundError: Found no valid file for the classes inception-v3. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [17]:

X = df.drop('Exited',axis=1)
y = df.Exited

NameError: name 'df' is not defined